In [2]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import scipy.optimize as opt
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
from wrfpywind import data_preprocess as pp

## Read in the ensemble and observational data

In [3]:
wrfens = pp.fmt_wrfens_wspd(wrfout_headdir='../data/', 
                    wrfout_dirs=['28mp4lw4sw2lsm5pbl1cu/',  # Lee 2017
                                 '8mp1lw1sw2lsm1pbl1cu/',  # Draxl 2014a
                                 '8mp1lw1sw2lsm2pbl1cu/',  # Draxl 2014b
                                 '8mp1lw1sw2lsm2pbl3cu/',  # Vernon 2018
                                 '8mp4lw2sw2lsm5pbl3cu/',  # Optis 2021
                                 ],
                    wrfout_files=['ow_buoy_wrfout_d03_2020-02-05-2020-02-11',
                                #   'ow_buoy_wrfout_d03_2020-06-03-2020-06-09',
                                #   'ow_buoy_wrfout_d03_2020-07-01-2020-07-07',
                                #   'ow_buoy_wrfout_d03_2020-11-26-2020-12-02',
                                  ],
                    model_names=['Lee 2017', 'Draxl 2014a', 'Draxl 2014b', 'Veron 2018', 'Optis 2021'],
                    heights=[20, 40, 60, 80, 100, 120, 140, 160, 180, 200],
                    locations=['south', 'north']
                    )

/Users/jeffreysward/opt/anaconda3/envs/enspp/lib/python3.8/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'pressure' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/Users/jeffreysward/opt/anaconda3/envs/enspp/lib/python3.8/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'pressure' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/Users/jeffreysward/opt/anaconda3/envs/enspp/lib/python3.8/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'pressure' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/Users/jeffreysward/opt/anaconda3/envs/enspp/lib/python3.8/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'pressure' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/Users/jeffr

In [4]:
obs = pp.fmt_buoy_wspd(data_path='/Users/jeffreysward/Box Sync/01_Research/01_Renewable_Analysis/WRF_Solar_and_Wind/oshwind/wrfpywind/wrfpywind/data/nyserda_buoy/', 
                       south_dates_str='20190904_20210207', north_dates_str='20190812_20210207', 
                       heights=[20, 40, 60, 80, 100, 120, 140, 160, 180, 200],
                       start_date='01-01-2020', end_date='12-31-2020')

In [5]:
# y, x = np.histogram(obs, bins=45, normed=True)
xx = np.linspace(0,30,1000)
a, b, loc, scale = stats.truncnorm.fit(obs)  
pdf_truncnorm = stats.truncnorm.pdf(xx, a, b, loc, scale)  
fig, ax = plt.subplots(figsize=(7,5))

# fitted distribution
plt.plot(xx,pdf_truncnorm,'r-', lw=5, alpha=0.6, label='truncnorm pdf')
plt.plot(xx,pdf_truncnorm,'k-')
plt.hist(obs, bins= 45, density=True, label='Data')
ax.set_xlabel('Wind Speed, '+r'$x (m s^{-1})$',size = 15)
ax.set_ylabel('Frequency, '+r'$P(x)$',size=15)
ax.legend(loc='best', frameon=False)

RuntimeError: The data contains non-finite values.

In [ ]:
# Get data for only north buoy at 100m and format
wrfens = wrfens.sel(location='north', height=100)
wrfens = wrfens.T.to_pandas()
obs = obs.sel(location='north', height=100)
obs = obs.to_pandas()
obs = obs.dropna()

## Fit distributions to the observed wind speeds
Try gamma and trucated normal

In [ ]:
# y, x = np.histogram(obs, bins=45, normed=True)
xx = np.linspace(0, 30, 1000)
a,loc,scale = stats.gamma.fit(obs)  
pdf_gamma = stats.gamma.pdf(xx, a, loc, scale)  
fig, ax = plt.subplots(figsize=(7,5))

# fitted distribution
plt.plot(xx,pdf_gamma,'r-', lw=5, alpha=0.6, label='gamma pdf')
plt.plot(xx,pdf_gamma,'k-')
plt.hist(obs, bins= 45, density=True, label='North Buoy Data')
ax.set_xlabel('Wind Speed, '+r'$x (m s^{-1})$', size = 15)
ax.set_ylabel('Frequency, '+r'$P(x)$',size=15)
ax.legend(loc='best', frameon=False)